## import modules

In [1]:
import time
from collections import namedtuple

import numpy as np
import tensorflow as tf

## tokenizer
> convert the character to and from integers

In [38]:
with open('history.txt', 'r', encoding='utf-8') as f:
    text = f.read()
vocab = sorted(set(text))
vocab_to_int = {c: i for i,c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
encoded = np.array([vocab_to_int[c] for c in text], dtype=np.int32)

check out the first 100 characters

In [39]:
text[:100]

'\u3000\u3000卷一·五帝本纪第一\n\n\u3000\u3000黄帝者，少典之子，姓公孙，名曰轩辕。生而神灵，弱而能言，幼而徇齐，长而敦敏，成而聪明。轩辕之时，神农氏世衰。诸侯相侵伐，暴虐百姓，而神农氏弗能征。于是轩辕乃习用干戈，以征'

In [40]:
len(vocab)

4774

In [41]:
len(text)

644266

words frequency

In [42]:
len(text)/len(vocab)

134.95307917888562

encoded integers

In [43]:
encoded[:100]

array([  18,   18,  506,   30,    7,   98, 1137, 1776, 3008, 2879,   30,
          0,    0,   18,   18, 4695, 1137, 3153, 4770, 1021,  333,   70,
        937, 4770,  874,  323,  942, 4770,  566, 1750, 3965, 3995,   20,
       2545, 3155, 2756, 2331, 4770, 1218, 3155, 3226, 3669, 4770, 1153,
       3155, 1249, 4721, 4770, 4280, 3155, 1634, 1623, 4770, 1437, 3155,
       3175, 1702,   20, 3965, 3995,   70, 1692, 4770, 2756,  346, 2034,
         45, 3590,   20, 3755,  219, 2646,  220,  156, 4770, 1745, 3491,
       2615,  874, 4770, 3155, 2756,  346, 2034, 1206, 3226, 1245,   20,
         94, 1714, 3965, 3995,   67,   83, 2547, 1147, 1431, 4770,  143,
       1245], dtype=int32)

In [44]:
int_to_vocab

{0: '\n',
 1: "'",
 2: '.',
 3: '`',
 4: 'k',
 5: '{',
 6: '}',
 7: '·',
 8: 'Я',
 9: '‘',
 10: '’',
 11: '“',
 12: '”',
 13: '…',
 14: '┱',
 15: '■',
 16: '□',
 17: '◎',
 18: '\u3000',
 19: '、',
 20: '。',
 21: '々',
 22: '《',
 23: '》',
 24: '䌷',
 25: '䎬',
 26: '䜣',
 27: '䲡',
 28: '䴔',
 29: '䴖',
 30: '一',
 31: '丁',
 32: '七',
 33: '万',
 34: '丈',
 35: '三',
 36: '上',
 37: '下',
 38: '不',
 39: '与',
 40: '丐',
 41: '丑',
 42: '专',
 43: '且',
 44: '丕',
 45: '世',
 46: '丘',
 47: '丙',
 48: '业',
 49: '丛',
 50: '东',
 51: '丝',
 52: '丞',
 53: '两',
 54: '严',
 55: '丧',
 56: '个',
 57: '中',
 58: '丰',
 59: '临',
 60: '丸',
 61: '丹',
 62: '为',
 63: '主',
 64: '丽',
 65: '举',
 66: '乂',
 67: '乃',
 68: '久',
 69: '义',
 70: '之',
 71: '乌',
 72: '乍',
 73: '乎',
 74: '乏',
 75: '乐',
 76: '乔',
 77: '乖',
 78: '乘',
 79: '乙',
 80: '九',
 81: '乞',
 82: '也',
 83: '习',
 84: '乡',
 85: '书',
 86: '买',
 87: '乱',
 88: '乳',
 89: '乾',
 90: '予',
 91: '争',
 92: '事',
 93: '二',
 94: '于',
 95: '亏',
 96: '云',
 97: '互',
 98: '五',
 99: '井',
 100

## Making training mini-batchs

In [45]:
def get_batches(arr, batch_size, n_steps):
    '''
    Create a generator that returns batches of size
       batch_size x n_steps from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       n_steps: Number of sequence steps per batch
    '''
    characters_per_batch = batch_size * n_steps
    n_batches = len(arr)//characters_per_batch
    
    arr = arr[:characters_per_batch*n_batches]
    
    arr = np.reshape(arr,[batch_size,-1])
    
    for n in range(0, arr.shape[1], n_steps):
        x = arr[:, n:n+n_steps]
        y_temp = arr[:,n+1:n+n_steps+1]
        
        y = np.zeros(x.shape, dtype=x.dtype)
        y[:, :y_temp.shape[1]] = y_temp
        
        yield x, y

test the function

In [46]:
batchs = get_batches(encoded, 5, 10)
x, y = next(batchs)

In [47]:
print(x)
print(y)

[[  18   18  506   30    7   98 1137 1776 3008 2879]
 [   0   18   18   98 2002 1707 4770  376 1988   50]
 [4326   70  461 4770  143   50   59 3707 4770  487]
 [3155 3042 1681 1119   70 2043 4770  331  442 1269]
 [1626  133 2945 4770 4331 1050 2089   70 4350  143]]
[[  18  506   30    7   98 1137 1776 3008 2879   30]
 [  18   18   98 2002 1707 4770  376 1988   50   50]
 [  70  461 4770  143   50   59 3707 4770  487  708]
 [3042 1681 1119   70 2043 4770  331  442 1269 1681]
 [ 133 2945 4770 4331 1050 2089   70 4350  143  142]]


## Building the model

### inputs

In [48]:
def build_inputs(batch_size, num_steps):
    ''' Define placeholders for inputs, targets, and dropout 
    
        Arguments
        ---------
        batch_size: Batch size, number of sequences per batch
        num_steps: Number of sequence steps in a batch
        
    '''
    inputs = tf.placeholder(tf.int32, shape=[batch_size, num_steps], name='inputs')
    targets = tf.placeholder(tf.int32, shape=[batch_size, num_steps], name= 'targets')
    
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    return inputs, targets, keep_prob

### Lstm cell

In [49]:
def build_lstm(lstm_size, num_layers, batch_size, keep_prob):
    ''' Build LSTM cell.
    
        Arguments
        ---------
        keep_prob: Scalar tensor (tf.placeholder) for the dropout keep probability
        lstm_size: Size of the hidden layers in the LSTM cells
        num_layers: Number of LSTM layers
        batch_size: Batch size

    '''
    def build_cell(lstm_size, keep_prob):
        lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return drop
    
    cell = tf.contrib.rnn.MultiRNNCell([build_cell(lstm_size, keep_prob) for _ in range(num_layers)])
    init_state = cell.zero_state(batch_size, dtype=tf.float32)
    
    return cell, init_state

### RNN Output

In [50]:
def build_output(lstm_output, in_size, out_size):
    '''
        Build a softmax layer, return the softmax output and logits.
    
        Arguments
        ---------
        
        lstm_output: List of output tensors from the LSTM layer
        in_size: Size of the input tensor, for example, size of the LSTM cells
        out_size: Size of this softmax layer
    '''
    
    seq_output = tf.concat(lstm_output, axis=1)
    x = tf.reshape(seq_output, [-1, in_size])
    
    with tf.variable_scope('softmax'):
        softmax_w = tf.Variable(tf.truncated_normal([in_size,out_size],stddev=1.0))
        softmax_b = tf.Variable(tf.zeros(out_size))
    
    logits = tf.matmul(x, softmax_w) + softmax_b
    
    out = tf.nn.softmax(logits, name='predictions')
    
    return out, logits

## Training loss

In [51]:
def build_loss(logits, targets, lstm_size, num_classes):
    ''' Calculate the loss from the logits and the targets.
    
        Arguments
        ---------
        logits: Logits from final fully connected layer
        targets: Targets for supervised learning
        lstm_size: Number of LSTM hidden units
        num_classes: Number of classes in targets
        
    '''
    y_one_hot = tf.one_hot(targets,num_classes)
    y_shaped = tf.reshape(y_one_hot, [-1, num_classes])
    
    loss = tf.nn.softmax_cross_entropy_with_logits(labels=y_shaped, logits=logits)
    loss = tf.reduce_mean(loss, name='loss')
    
    return loss

### Optimizer

In [52]:
def build_optimizer(loss, learning_rate, grad_clip):
    ''' Build optmizer for training, using gradient clipping.
    
        Arguments:
        loss: Network loss
        learning_rate: Learning rate for optimizer
    
    '''
    
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    return optimizer

### Build the network

In [53]:
class CharRNN:
    def __init__(self, num_classes, batch_size=64, num_steps=50,
                lstm_size=128, num_layers=2, learning_rate=0.001,
                grad_clip=5, sampling=False):
        if sampling == True:
            batch_size, num_steps = 1, 1
        else:
            batch_size, num_steps = batch_size, num_steps
        
        tf.reset_default_graph()
        
        self.inputs, self.targets, self.keep_prob = build_inputs(batch_size, num_steps)
        
        cell, self.initial_state = build_lstm(lstm_size, num_layers, batch_size, self.keep_prob)
        
        x_one_hot = tf.one_hot(self.inputs, num_classes)
        
        outputs, state = tf.nn.dynamic_rnn(cell, x_one_hot,initial_state=self.initial_state)
        self.final_state = state
        
        self.prediction, self.logits = build_output(outputs, lstm_size, num_classes)
        
        self.loss = build_loss(self.logits, self.targets, lstm_size, num_classes)
        self.optimizer = build_optimizer(self.loss, learning_rate, grad_clip)

### Hyperparameters

In [54]:
batch_size = 20         # Sequences per batch
num_steps = 50          # Number of sequence steps per batch
lstm_size = 128         # Size of hidden layers in LSTMs
num_layers = 2          # Number of LSTM layers
learning_rate = 0.01    # Learning rate
keep_prob = 0.5         # Dropout keep probability

### Training

In [55]:
epochs = 3
# Print losses every N interations
print_every_n = 50

# Save every N iterations
save_every_n = 200

model = CharRNN(len(vocab), batch_size=batch_size, num_steps=num_steps,
                lstm_size=lstm_size, num_layers=num_layers, 
                learning_rate=learning_rate)

saver = tf.train.Saver(max_to_keep=100)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Use the line below to load a checkpoint and resume training
    #saver.restore(sess, 'checkpoints/______.ckpt')
    counter = 0
    for e in range(epochs):
        # Train network
        new_state = sess.run(model.initial_state)
        loss = 0
        for x, y in get_batches(encoded, batch_size, num_steps):
            counter += 1
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state}
            batch_loss, new_state, _ = sess.run([model.loss, 
                                                 model.final_state, 
                                                 model.optimizer], 
                                                 feed_dict=feed)
            if (counter % print_every_n == 0):
                end = time.time()
                print('Epoch: {}/{}... '.format(e+1, epochs),
                      'Training Step: {}... '.format(counter),
                      'Training loss: {:.4f}... '.format(batch_loss),
                      '{:.4f} sec/batch'.format((end-start)))
        
            if (counter % save_every_n == 0):
                saver.save(sess, "west_checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))
    
    saver.save(sess, "west_checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))

Epoch: 1/3...  Training Step: 50...  Training loss: 6.0492...  0.1219 sec/batch
Epoch: 1/3...  Training Step: 100...  Training loss: 5.7839...  0.1193 sec/batch
Epoch: 1/3...  Training Step: 150...  Training loss: 5.6773...  0.1216 sec/batch
Epoch: 1/3...  Training Step: 200...  Training loss: 5.5752...  0.1219 sec/batch
Epoch: 1/3...  Training Step: 250...  Training loss: 5.5011...  0.1200 sec/batch
Epoch: 1/3...  Training Step: 300...  Training loss: 5.5243...  0.1202 sec/batch
Epoch: 1/3...  Training Step: 350...  Training loss: 5.4062...  0.1189 sec/batch
Epoch: 1/3...  Training Step: 400...  Training loss: 5.4532...  0.1169 sec/batch
Epoch: 1/3...  Training Step: 450...  Training loss: 5.4434...  0.1222 sec/batch
Epoch: 1/3...  Training Step: 500...  Training loss: 5.4772...  0.1197 sec/batch
Epoch: 1/3...  Training Step: 550...  Training loss: 5.1830...  0.1196 sec/batch
Epoch: 1/3...  Training Step: 600...  Training loss: 5.1083...  0.1220 sec/batch
Epoch: 2/3...  Training Step:

### Saved checkpoints

In [56]:
tf.train.get_checkpoint_state('west_checkpoints')

model_checkpoint_path: "west_checkpoints/i1932_l128.ckpt"
all_model_checkpoint_paths: "west_checkpoints/i200_l128.ckpt"
all_model_checkpoint_paths: "west_checkpoints/i400_l128.ckpt"
all_model_checkpoint_paths: "west_checkpoints/i600_l128.ckpt"
all_model_checkpoint_paths: "west_checkpoints/i800_l128.ckpt"
all_model_checkpoint_paths: "west_checkpoints/i1000_l128.ckpt"
all_model_checkpoint_paths: "west_checkpoints/i1200_l128.ckpt"
all_model_checkpoint_paths: "west_checkpoints/i1400_l128.ckpt"
all_model_checkpoint_paths: "west_checkpoints/i1600_l128.ckpt"
all_model_checkpoint_paths: "west_checkpoints/i1800_l128.ckpt"
all_model_checkpoint_paths: "west_checkpoints/i1932_l128.ckpt"

### Sampling

In [57]:
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [58]:
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime="The "):
    samples = [c for c in prime]
    model = CharRNN(len(vocab), lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            x[0,0] = vocab_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

        c = pick_top_n(preds, len(vocab))
        samples.append(int_to_vocab[c])

        for i in range(n_samples):
            x[0,0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])
        
    return ''.join(samples)

In [59]:
tf.train.latest_checkpoint('west_checkpoints')

'west_checkpoints/i1932_l128.ckpt'

In [36]:
checkpoint = tf.train.latest_checkpoint('west_checkpoints')
samp = sample(checkpoint, 2000, lstm_size, len(vocab), prime="孙")
print(samp)

孙八戒道：“
你是这个小妖，你怎么不敢？”八戒道：“这和尚也，不知不得了
我，却不是我师兄，我们不曾见他。”行者道：“我们是他的，
你们不曾说得，这等说不知，这个是他，不是他，不知是这里是，
不是不得，不知，不知好，你不见，他就在我，不知我们，这个
是个大圣，不能是个。”行者笑道：“师父！”行者道：“
你怎么不好！你是那怪，我们也是那妖精，他怎么不知！”那
妖精道：“你是那里？”行者道：“这里是这般，是我们去。
”八戒道：“我是我，这等有个人的，只有那里？”那女子道：
“你不知，那个是他，不是我，我们是我，不曾打杀他，不知
不敢，我也不是，只是不是！”那龙婆道：“你们是那怪，
我们是一个妖精！”行者道：“你们不知！”八戒道：“哥哥，
我是这里来？”行者道：“这里来，你不曾见你，我不是
你不知，我是你不曾见我，只是有些儿，我也不知不敢
，他是个和我去也。”行者笑道：“这里，好，好，好！”那
妖精不敢，只见那妖魔，不知，不敢，不知，只得不得。那
妖魔，不知，又不知，只见那山里见了那怪，又听见一般，
又见了一声。那呆子道：“我们去了。”唐僧道：“你是个
妖精。”行者道：“你是他的的，是你不知！我们是这里是
个？我们怎生得？我等不曾说得我，不知不知，你不知这等，
你不是他，我等怎么不是。”唐僧道：“这个不曾说，我不曾
不得，却说我的，我们不是他的人，却是个大圣，只是那里来。”
行者道：“哥哥啊！这个好！那里来？”行者道：“哥啊，不知
我的人，只是有一个的和尚，我们去了。”唐僧道：“你怎么
的？”八戒道：“这个是一个小和尚的和尚，我不要去，不好，
不敢，不知不敢，他不知是那个人儿。他这等，是他一般，你
看他不曾，我们有些，不知不曾见，只得不得他！”
那呆子一个个小龙来，不知，不敢说了，那里去了。八戒
道：“哥哥，我是这个个和尚，你们不是他，我们有个和尚，
不知，不好，只是不得，不曾不得，只是那里去了？”八戒道：“
这里有人，不知不知，不敢，我们是个和尚的，我不曾见了，
却说是他，不得不敢，我们是那里去。”八戒道：“师父，
他不知不得，只是是那妖精。”行者道：“你们这般，我
们不敢了。”那怪道：“我不知，我们不是，你是个和尚，不敢
打。”行者道：“你这个不知，不知是你不知！”那怪道：“这里
不是。”行者道：“你这个是个小龙，你不敢说，我等我去了，你
们怎么得？”行者道：“我们，

In [37]:
checkpoint = tf.train.latest_checkpoint('west_checkpoints')
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="我")
print(samp)

我的一声，
只听是一个小龙，一声不题。却说那八戒道：“你不曾
得了！”那龙圣又见了，那怪道：“你这个好的！”八戒笑
道：“这个是我师父，你这般，不是我师父！你怎么好！”唐僧道：“
这和尚也不曾得他！”那怪道：“这厮好的，不知，我怎么不
见？”行者道：“你们这里去，我们是一个个小和尚也。”八戒
道：“这厮是你不曾打，我不曾打了一个，不是，你却有一个
的？”八戒道：“你不知是你的人，我怎么说，你是个妖精的
？”行者道：“我们不知，我是这里是我们的？”行者道：“
你不得？”八戒道：“这里，好是我，不知好！你看他不曾，
我们是那个和尚。他不是，我们有什么妖精，你不知，这里是那
里来？我不敢不曾，不是不得，却不知我不知。”行者道：“我
的，是那妖怪，不敢不知，只是不知，我是这般，只是不曾，
我怎么不知？”行者道：“不好，我是你不知，你是那和尚的？
”唐僧道：“哥哥，他不敢，你不敢不知，不知我师父，你
不敢，他也不是不知？”八戒道：“不好，我们是我的和尚，
不知是个是人，你是这里去了？”八戒道：“哥啊，你怎么
不知！”行者道：“你这等不知！”八戒道：“你不知，不是我
的，是那个人，我不知不得，我们不知，不知，我们是我师父，
你不知是他不知，不知不知不知，不曾是那妖怪。”行者
道：“你是这里？”那怪道：“哥啊，你是他的人的的，你不是

我的？”八戒笑道：“你不知不是，你不敢，我们不是
他。”行者道：“这个是他是那里去？”行者道：“你是
那里？”行者道：“我们有一个，不知我师父，不知，你怎么
不敢？”行者道：“我们去了！”行者道：“这个小龙，不是我
师父的，却怎么是？这般是你，你怎么好！你这里不敢，你不知这
般有些？”行者道：“你不是我们！他这个不敢，我等说，他是我的
一个的儿，不知我们不知不得，只有一个，不敢，只是一个一
个小妖，不知是个个人儿？”八戒道：“哥啊，他不得
不知。”八戒道：“哥啊，这般是他，这里有人，我们
不敢不曾，不敢说，你们是那妖怪！这个不是，你不知不知，不是他
不敢不知，你怎么不知？”八戒道：“你是这个和尚，他就不是，

你是这个妖精！他是这里来的，好！”八戒道：“这般
是个和尚！你怎么不知！”那妖魔，一齐跳出来，只见一
座山里，那里来见了。那妖精不见。行者笑道：“哥哥，我的
一个，我们是我，不曾说了，我是我的，我也不是，我们不敢打
了，却不是个和尚的？”行者

In [60]:
checkpoint = tf.train.latest_checkpoint('west_checkpoints')
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="我")
print(samp)

我之，不能不能，以故其为人。今王不得以王之，而为秦王也，故为其不可以以为王之事之也。

　　臣闻王不如秦王之所与秦也，不能与楚之所以不可而以为王之，而王之不能也；而王之不可以以其功。今楚王之国，秦王之不能不可以以之，不得其国也。

　　秦之所以以秦之彊也，而秦之不能也。秦之兵而秦，秦之兵而伐齐，而楚不能与楚之秦，以为楚之秦。”楚王曰：“臣之所以以王以其为之。”于是遂至，乃以王为王。王曰：“臣闻之，乃使人不听，乃以为秦王也。臣为天下之以不肖，以此为国之所以也。夫魏之以秦，则楚之所以为秦，则秦之彊秦而秦之秦也。今齐之秦，而韩、赵、韩、魏、魏、楚、秦、魏、秦、韩、魏，以为秦，不能以秦也，以王为天下之，不敢以为秦，不能为秦之所以以其所不能以其国，不可得。今秦以兵之国，而楚必以为之矣。

　　楚，秦以为魏、秦也，以其地为秦。魏之秦也。秦必与秦、韩、魏、魏、楚之地，秦必以秦兵，秦王不能得之，以秦王以之，则王不得之，不可以不敢以为之。今楚以兵，秦王以兵以秦之地而以其地而不能不听。秦必为齐王之国，则楚之所不能能之秦，不可也。”王乃以其子以以为诸侯，以为大夫。秦之秦之国以为楚，不敢与齐之地。秦王不听，以为秦。

　　秦楚王不能，秦王不敢得。王不敢以为王，不敢为秦。秦必以为魏王，以为秦。楚王，秦必以为秦，而不得秦。秦以秦秦之国，则韩、魏、魏、秦，楚之彊，秦王之不得也。

　　魏之彊之地，则楚之彊，秦必之楚，不得为秦而以为王，则秦之彊，秦之彊，秦之兵之地也，不如秦之地也。王不敢不能以以为王之功矣。

　　魏王之以秦之秦，则秦以彊楚，而楚秦以兵之秦而楚，不能为秦，不能得其，不能以其所以以为国也。臣之为王，不能以兵之，必以秦为王而以为之，不可以为秦矣。”秦王曰：“秦不可与秦，以王之为秦之，而以秦之为秦，不可以兵。

　　“王曰：”秦不听，而秦王之为之，以其彊，不得而为王，不能以秦也。秦必不敢以为楚也。王之以为王之以而之而不能以秦，则秦不可得之。秦王必得齐，而王之不可也，则秦之所以以为秦也。臣之所得，则魏不得，则王以兵之兵也。”

　　赵王闻其后，乃使使人将，与赵将兵击之。秦攻齐，楚兵破赵。秦使人击赵王之兵，破赵军，破之。楚兵击魏、赵、秦、韩等，以兵破秦军，击秦，击秦军，虏赵王、秦、魏，破之。魏王卒卒，谥为公公。

　　二十一年，楚王伐魏，败之。十二年卒，子王立。

　　二年，吴王卒，子平王立

In [61]:
checkpoint = tf.train.latest_checkpoint('west_checkpoints')
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="情")
print(samp)

情，其为天下之，而其不可。”

　　秦之有所言，不得以为天子，故不听，以其为人，以天下无能，其以故其人也。

　　“夫王曰‘秦之不得也，不能能得其不可以以为也，不可以以为也。今君之不如不肖而以其所为也！

　　“”于是曰：“秦以为之。”王曰：“臣以为天下之也。”于是秦王乃以为其王。

　　楚将王，楚王以为秦，以为秦王。

　　魏襄王曰：

　　楚王以其，秦之秦之彊，而秦攻齐、韩、楚、赵、秦、魏、魏。魏秦攻楚，楚王不能为也。”楚王曰：“楚不能得之，不敢与楚之王，而不敢与秦之兵，不可得之也。”

　　赵王乃立太子、魏王。楚王乃以为太子，乃为太子。楚王不敢与秦。楚使楚、魏、韩、魏皆以为王，为王。

　　汉王元年，秦王为秦王，使使人使将将击魏、楚。

　　楚，王为魏王。

　　项梁王，使使使王使使，遂击楚军。项梁以兵为楚王，使使使大将军，将军，斩首万户，以兵击项羽，斩首万余人。

　　项梁以兵，项王为汉将军，将军将将军军，将军军，军军击，军击楚，破，军军将，虏项羽，军将兵。项梁以攻魏、楚、楚、魏、魏、魏，破楚，攻魏之，以兵击韩、魏军，击楚，败秦军军。楚将兵军，攻楚，以兵击项梁军，斩首六十万，破破楚军。十一年，秦击魏，破秦，破魏。楚王败之。赵王遂击楚，击秦。秦攻韩，秦击秦，破魏之兵。

　　汉王既击楚，楚军破之。秦王与秦兵，击秦军。赵攻韩，秦攻魏。

　　楚王卒，以兵西兵。

　　楚王卒，子太子立，为魏王，是为王。

　　十四年，秦伐楚，破之。二十四年，吴王卒。吴王卒，子平王立。

　　秦王曰：“魏王之为之，不如秦之，不能与秦，秦不得秦。秦以齐秦之兵，秦不能得之，而楚之不听也。”魏王以之，以为楚王。秦王不听。秦使王曰：“王不得，以秦之兵。秦王以秦之彊，则秦之地而以秦之地也。

　　秦之秦而秦之秦之秦，必不可也。”王曰：“王以其国，必不可以其所以为之。秦不能不能以为秦。”楚王曰：“王闻其所以不敢不得，以为天下，以为秦也。夫魏王不听也，则王之以秦，秦之不得，不能不敢与秦也。”秦王曰：“臣以为王也。”

　　太后曰：“王不能能得秦，必不可以为君之也。今秦之以秦，则秦不如不听。秦以秦兵，秦王不得，以为王，以为秦之王，而秦之以为秦。秦王不能不可也；而秦不敢与秦，不如不可也！”秦王乃使秦使人之秦，以为魏。秦之以齐王、秦之地，齐人、赵王、赵之以为秦，秦王以兵，以为秦之地，而以其为秦王而与秦

In [63]:
checkpoint = tf.train.latest_checkpoint('west_checkpoints')
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="赵")
print(samp)

赵，以其国不听，乃以为大夫，以为王。今天下之所以以秦王，为之，以秦之为秦，不能为也。”楚王乃使人与赵之王、魏之兵，而与韩信之，而破之。

　　秦王闻之，以秦王与魏、魏之攻齐，魏王之为魏，以其兵以为王。

　　楚王，以为王，乃使人为楚王，遂与秦兵，败秦，败魏，与秦攻秦。楚、楚、魏、秦、赵、魏、魏、赵、魏、魏、赵、赵、魏、魏、魏、魏、赵、赵、韩、赵、魏、韩、魏、楚、秦、魏、秦，破楚军，楚兵，击楚。赵攻魏、秦、楚、赵、魏，以魏之秦。赵王以为秦，与韩、魏之兵，破之。赵将军赵，破之，以为秦。

　　楚楚王，使使使人与吴王与魏、魏王之兵，楚王以其为王。秦以其兵，与赵、魏、韩、楚、秦，攻楚，以为魏、秦、魏之地。魏楚以为秦、魏、魏、秦、魏、魏、楚、秦、韩、韩、魏、赵、魏、赵、秦、秦、魏、魏、赵、魏、魏、楚、楚、赵、秦，秦兵之地而以其地，而秦兵之兵。秦王以为王。秦王之以为魏，秦王为王之国，而楚王不可以以其事之矣。秦以秦之兵，楚之彊，而秦之以秦之也也。今秦之秦也，秦以楚之国也。楚王以为齐王。秦秦之国而齐王之秦也，秦王之以彊秦也，而秦之彊，则韩、魏之所以为秦也。今秦之兵而以齐秦之地也，不得为秦，则不得不可。今王不得，而秦必得秦而以兵，秦不敢与秦，秦以之而不能为之矣。今王以其所与之国，而以韩王之不能以兵之地，不敢不可以为王也。今王不能与齐王，必以秦兵。楚王之所得，不敢得王，则王以为秦，必不能得，不能不敢不可也。”楚王曰：“王不能不听，则秦不能得而不能不敢，而王之所得，不可以为之矣。”王以兵，使王之为王。

　　秦王曰：“魏之不可不敢以为王，以其国之地，不得其也。

　　“秦、魏，秦之彊国，不敢以秦。秦之彊之地而而韩王之以秦之地也。今王不能不能为王，不可以故秦。魏王之为秦，则王之不敢以其彊，秦必与魏、赵、魏、赵，魏以楚，而韩之秦也也。今楚，王不可以为也。今魏王而与韩、魏，楚不可以以为秦。秦王不能以秦，而楚之王，不能得其所不能以为楚，必得王不听也。今秦王以为王之所与秦，秦之秦之兵而伐韩，秦必与齐之秦之秦也。”楚王乃使楚使秦兵。魏将攻魏，魏之秦而攻楚，以为楚。秦使秦之兵，而秦使秦兵之兵，秦之地之秦，秦王以齐、秦、赵之所以攻齐也，而王以为秦王之也，以秦之秦之地，而秦必不听，则秦以为秦之秦，秦之彊地也。秦王以为魏，不能以秦之也。今王而之，而韩之秦，秦之彊国之兵也；不可而不能之也，故以其国以为天下之也

In [64]:
checkpoint = tf.train.latest_checkpoint('west_checkpoints')
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="故")
print(samp)

故。”王曰：“臣为天下，何以有为也？”王曰：“吾以为王。”王曰：“秦以秦之国而不可以为也。”秦王曰：“王不得，无能以为之。”王曰：“寡人之所以以天子以其所与也，故为其人之也。夫王不可以故，不敢得为，不敢不得而可以为。”于是王不敢见。王不听，乃使人不听，以为其所以为。王乃使王之子，乃以其为太子，为之而与王之，以为秦。王乃使其子、秦、韩、秦、魏皆、魏、秦、魏、魏王，与楚、韩、赵、魏、赵、赵、魏之兵，破之于楚，而与魏王之地以秦之，而王不敢与秦，而与魏王不可以，以兵为秦。秦之以齐王之以秦王之以秦之兵之所以其彊，秦不敢得秦也。王以其不敢不可也，不敢不敢与魏，不如以为魏。今楚之兵，而韩、魏、魏之所以与秦，不能与魏，而以为韩王之为之也，以为秦之所以为秦之以秦，而王之不听矣。王之为楚，不敢得秦也。

　　秦王之所与秦，楚不可以为秦矣，则楚王而以秦，不能以为楚，以王以其彊秦，秦以之秦。秦之彊秦之国也。”王之，以秦为魏王，而王不能为，不敢以兵，而与齐之秦，以兵兵，而攻楚，不能得其，以兵而伐楚，以楚为魏。秦之不可以秦之也。”王曰：“王何也，王之不敢与齐王之，不如秦而秦，以其兵不能为，不能不可以不可以不可。今秦以秦之秦，不能不得，必与秦、秦、魏以为齐、魏之秦，不如以其兵而以兵之秦而秦之兵，秦以兵不得而不可。今秦王以秦之地，则秦必攻之。楚不可不得，则秦之兵也，而以其国不能与魏之，秦必与秦之兵也，不能不听，而王不能以兵之以以其国不能为也。

　　“魏王以秦，秦王以兵以为秦，则王不可也。秦王之不能以秦之，必以其兵之兵，以为秦之所以不能不能，秦不能与魏。秦必不可以不听，不如不敢以为楚。”秦以秦兵，与秦兵，攻魏，秦攻韩、韩，魏将之，破之，秦兵败之。

　　魏楚王，秦兵败，与赵王以为楚，与魏，攻楚之地，以楚军为秦军，攻魏，与赵攻魏，秦攻魏，秦攻楚，破之，破之，以为秦将军，与韩、楚、魏、魏、楚，攻赵，破赵。秦攻魏，秦败楚，楚败秦，楚王不听。楚将，以秦攻赵，楚使王之，与魏王与魏攻楚，与秦军败之。秦将兵攻赵，破秦，败楚，败之。

　　二年，魏王与楚将军，攻楚、魏、赵、魏、赵，以兵攻韩、赵。赵王乃与魏王、魏王，以为秦王，与楚将将军军而与赵，败秦，击楚将军，破赵军，击赵军，虏秦军，击魏、赵、魏、赵王之，以军西，斩首虏万人，虏其王将军。十六岁，与韩、魏，击魏，将攻魏，破魏。赵王与秦将兵击韩、魏、魏，与楚、楚、楚，